In [2]:
import os
import sys
import seawater as sw
from scipy import interpolate as interp

sys.path.append("..")
sys.path.append("../scripts")
sys.path.append("../src/iw_model")
sys.path.append("../src/iw_inverse")
sys.path.append("../src/misc")

from iw_invert import InternalWaveInversion
from iw_plots import *
from extract import *


In [14]:
"""
Read data and compute means
"""
path = os.path.join("..","matlab",filename)

table = read_table(path)
N2 = extract_node(table,'N2')
T  = extract_node(table,'T')
S  = extract_node(table,'S')
Z  = extract_node(table,'z').flatten()

#Compute the mean stratification
DSL = slice(0,70) # depth slice
N2_mean = np.mean(N2,axis=(0,2,3))
T_mean  = np.mean(T,axis=(0,2,3))
S_mean  = np.mean(S,axis=(0,2,3))
#R_mean  = sw.eos80.dens(S_mean[DSL],T_mean[DSL],Z[DSL])
R_mean  = sw.eos80.dens(S_mean[DSL],T_mean[DSL],0)


#Define a slice
sl = (slice(0,-1),DSL,slice(0,-1),slice(0,-1))

#Subsample matrices
TP = T[sl]
SP = S[sl]
NP = N2[sl]

TIME   = extract_node(table,'time').flatten()[sl[0]]
LAT    = extract_node(table,'lat')[:,0][sl[2]]
LON    = extract_node(table,'lon')[0,:][sl[3]]
DEPTH  = extract_node(table,'z').flatten()[sl[1]]

#Form a coordinate matrix
AX = np.array(list(itertools.product(*[TIME,DEPTH,LAT,LON])),dtype=[('t',float),('z',float),('x',float),('y',float)])
AX = AX.reshape(TP.shape)

#Compute Density Perturbations from state variables
#RP = sw.eos80.dens(SP,TP,AX['z'])
RP = sw.eos80.dens(SP,TP,0)

#Create a function that is depth as a function of the mean density
zofr = interp.InterpolatedUnivariateSpline(R_mean,Z[DSL])
DZ   = zofr(RP) - AX['z']


#Demean vertical profiles
DZB = np.mean(DZ,axis=(0,2,3))
for i in range(DZ.shape[0]-1):
    for j in range(DZ.shape[2]-1):
        for k in range(DZ.shape[3]-1):
              DZ[i,:,j,k] = DZ[i,:,j,k] - DZB




#Place an M2 band pass filter
def bandpass_filter(center,half_width,fs,order=5):
    nyq = fs*0.5
    lo = (center-half_width/nyq)
    hi = (center+half_width/nyq)
    b, a = sig.butter(order, [lo, hi], btype='band')
    return b,a

M2  = .0805 
bw  = .02
b,a = bandpass_filter(M2,bw,1)
DZ = sig.lfilter(b,a,DZ,axis=0)



In [15]:
"""
Modal Decomposition
"""
%load_ext autoreload
%autoreload
#Set up dataspace
df = pd.DataFrame({"dz"    : DZ[0,:,0,0].flatten(),
                   "z"     : AX[0,:,0,0]["z"].flatten(),
                   "time"  : 0,
                   "x"     : 0,
                   "y"     : 0
                    })

#Interpolate N2 with regular grid spacing for modes.
n2ofz = interp.InterpolatedUnivariateSpline(Z[DSL],N2_mean[DSL])
Z_even = np.linspace(Z[0],Z[70],100)


#Set up parameter space
FREQS = [.0805/3600]
MODES = np.arange(1,4)
iwi = InternalWaveInversion(df,FREQS,MODES,[0],Z_even,n2ofz(Z_even),tc='dz',mean=False)

iwi.pinvert()

#Apply adjoint for every depth profile to get modal amplitudes
amps = np.ndarray(shape=DZ[:,0,:,:].shape,dtype=[("m1",float),("m2",float),("m3",float)])
for i in range(DZ.shape[0]-1):
    for j in range(DZ.shape[2]-1):
        for k in range(DZ.shape[3]-1):
            v = iwi.hdag @ DZ[i,:,j,k]
            amps[i,j,k]=tuple(v[3:])
            
        
#df['dz_hat'] = iwi.H @ np.concatenate([iwi.ps['a'],iwi.ps['b']]).T


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
settin attrs
22.80969970703125


In [16]:
1/(iwi.ps['hwvn']*1000) 

0    128.134465+0.000000j
1     57.228416+0.000000j
2     30.072570+0.000000j
Name: hwvn, dtype: complex128

In [17]:
%matplotlib widget
for m in ['m1','m2','m3']:
    plt.plot(amps[0:500,0,0][m],label=m)
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
import matplotlib.animation as animation
from IPython.display import HTML
fig,ax = plt.subplots()
p = ax.contourf(amps[0,:,:]['m2'])

def update(frame):
    ax.contourf(amps[frame,:,:]['m2'],levels=20,cmap='seismic')
    return p,
    

#Compile Animation
ani = animation.FuncAnimation(fig, update,frames=np.arange(0,amps.shape[0],30),blit=False)

ani.save("foo.mp4")



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [147]:
f,ax = plt.subplots()
p = ax.contourf(amps[0:100,:,5]['m3'],levels=50,cmap='seismic')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [150]:
%matplotlib widget
stacked = amps[:,5,:].T.reshape(amps.shape[2]*amps.shape[0],-1)

f1,pxx1 = sig.welch(stacked.flatten()['m1'],fs=1/2,nperseg=11)
f2,pxx2 = sig.welch(stacked.flatten()['m2'],fs=1/2,nperseg=11)
f3,pxx3 = sig.welch(stacked.flatten()['m3'],fs=1/2,nperseg=11)
sl = slice(0,1000)
plt.plot(f1[sl],pxx1[sl])
plt.plot(f2[sl],pxx2[sl])
plt.plot(f3[sl],pxx3[sl])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …